Section 1/ Option 1- pulling the info from the API

In [1]:
import requests

# Base URL for the DnD 5e API
BASE_URL = "https://www.dnd5eapi.co"

def get_monsters():
    endpoint = "/api/monsters"
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        data = response.json()
        return data['results']  # List of monsters with 'name' and 'url'
    else:
        raise Exception(f"Failed to fetch monsters: {response.status_code}")

monsters = get_monsters()
print(f"Total monsters fetched: {len(monsters)}")


Total monsters fetched: 334


In [2]:
import time

def get_monster_details(monster_url):
    response = requests.get(BASE_URL + monster_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch {monster_url}: {response.status_code}")
        return None

# Collect all detailed monster data
detailed_monsters = []
for monster in monsters:
    details = get_monster_details(monster['url'])
    if details:
        detailed_monsters.append(details)
    time.sleep(0.1)  # To be polite to the API server

print(f"Detailed monsters fetched: {len(detailed_monsters)}")


Detailed monsters fetched: 334


In [3]:
import pandas as pd

# Function to extract required fields from each monster's data
def extract_monster_data(monster):
    data = {}
    # Simple fields
    data['name'] = monster.get('name')
    data['size'] = monster.get('size')
    data['type'] = monster.get('type')
    data['alignment'] = monster.get('alignment')
    data['hit_points'] = monster.get('hit_points')
    data['languages'] = monster.get('languages')
    data['challenge_rating'] = monster.get('challenge_rating')
    data['proficiency_bonus'] = monster.get('proficiency_bonus')
    data['xp'] = monster.get('xp')
    
    # Armor Class
    armor_classes = monster.get('armor_class', [])
    # We'll concatenate types and values
    ac_list = [f"{ac['type']}:{ac['value']}" for ac in armor_classes]
    data['armor_class'] = "; ".join(ac_list)
    
    # Speed
    speed = monster.get('speed', {})
    # Concatenate available speeds
    speeds = [f"{k}:{v}" for k, v in speed.items()]
    data['speed'] = "; ".join(speeds)
    
    # Attributes
    abilities = ['strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma']
    for ability in abilities:
        data[ability] = monster.get(ability)
    
    # Proficiencies
    proficiencies = monster.get('proficiencies', [])
    # Concatenate proficiency names and values
    prof_list = [f"{prof['proficiency']['name']}:{prof['value']}" for prof in proficiencies]
    data['proficiencies'] = "; ".join(prof_list)
    
    # Damage Vulnerabilities, Resistances, Immunities
    data['damage_vulnerabilities'] = "; ".join(monster.get('damage_vulnerabilities', []))
    data['damage_resistances'] = "; ".join(monster.get('damage_resistances', []))
    data['damage_immunities'] = "; ".join(monster.get('damage_immunities', []))
    
    # Condition Immunities
    condition_immunities = monster.get('condition_immunities', [])
    # Concatenate condition names
    cond_immunities = [cond['name'] for cond in condition_immunities]
    data['condition_immunities'] = "; ".join(cond_immunities)
    
    # Senses
    senses = monster.get('senses', {})
    # Concatenate senses and their values
    senses_list = [f"{k}:{v}" for k, v in senses.items()]
    data['senses'] = "; ".join(senses_list)
    
    # Special Abilities
    special_abilities = monster.get('special_abilities', [])
    # Concatenate special ability names
    special_abilities_list = [sa['name'] for sa in special_abilities]
    data['special_abilities'] = "; ".join(special_abilities_list)
    
    return data

# Extract data for all monsters
monsters_data = [extract_monster_data(monster) for monster in detailed_monsters]

# Create DataFrame
df = pd.DataFrame(monsters_data)

# Display the first few rows
print(df.head())


                 name    size        type      alignment  hit_points  \
0             Aboleth   Large  aberration    lawful evil         135   
1             Acolyte  Medium    humanoid  any alignment           9   
2  Adult Black Dragon    Huge      dragon   chaotic evil         195   
3   Adult Blue Dragon    Huge      dragon    lawful evil         225   
4  Adult Brass Dragon    Huge      dragon   chaotic good         172   

                           languages  challenge_rating  proficiency_bonus  \
0     Deep Speech, telepathy 120 ft.             10.00                  4   
1  any one language (usually Common)              0.25                  2   
2                   Common, Draconic             14.00                  5   
3                   Common, Draconic             16.00                  5   
4                   Common, Draconic             13.00                  5   

      xp armor_class  ... intelligence  wisdom  charisma  \
0   5900  natural:17  ...           18      

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Select features and target
# Exclude 'challenge_rating' from features
features = df.drop(['challenge_rating', 'name'], axis=1)  # 'name' is typically not useful for prediction
target = df['challenge_rating']

# Handle missing values if any
features = features.fillna('')  # Simple strategy; you can choose more sophisticated methods

# Identify categorical and numerical columns
# For simplicity, let's assume:
# - Numerical columns: strength, dexterity, constitution, intelligence, wisdom, charisma, hit_points, proficiency_bonus, xp
# - Categorical columns: size, type, alignment, armor_class, speed, proficiencies, damage_vulnerabilities, damage_resistances, damage_immunities, condition_immunities, senses, languages, special_abilities

numerical_features = ['strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma', 'hit_points', 'proficiency_bonus', 'xp']
categorical_features = [col for col in features.columns if col not in numerical_features]



In [5]:
# Define preprocessing for numerical and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create preprocessing and training pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

Mean Squared Error: 3.122401891719053
R² Score: 0.9271784382239454


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Update the pipeline with a Random Forest Regressor
model_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Define hyperparameters for tuning
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5],
}

# Setup GridSearch
grid_search = GridSearchCV(model_rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Train the model
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV MSE: {-grid_search.best_score_}")

# Predict on test set with the best estimator
y_pred_rf = grid_search.best_estimator_.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest Mean Squared Error: {mse_rf}")
print(f"Random Forest R² Score: {r2_rf}")


Best parameters: {'regressor__max_depth': None, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
Best CV MSE: 0.05684622434049615
Random Forest Mean Squared Error: 0.7941075209888062
Random Forest R² Score: 0.9814795942668744


In [11]:
def predict_challenge_rating(monster_attributes):
    """
    monster_attributes: dict containing the same keys as features columns
    """
    # Convert to DataFrame
    input_df = pd.DataFrame([monster_attributes])
    
    # Predict
    predicted_cr = grid_search.best_estimator_.predict(input_df)
    return predicted_cr[0]

# Example usage
new_monster = {
    'size': 'Medium',
    'type': 'dragon',
    'alignment': 'chaotic evil',
    'armor_class': 'natural:19',
    'speed': 'walk:40; fly:80',
    'strength': 27,
    'dexterity': 10,
    'constitution': 25,
    'intelligence': 16,
    'wisdom': 15,
    'charisma': 19,
    'hit_points': 546,
    'proficiency_bonus': 8,
    'xp': 390000,
    'proficiencies': 'Perception:10; Stealth:5',
    'damage_vulnerabilities': '',
    'damage_resistances': 'fire',
    'damage_immunities': '',
    'condition_immunities': 'frightened',
    'senses': 'darkvision:120; passive_perception:20',
    'languages': 'Common, Draconic',
    'special_abilities': 'Legendary Resistance; Frightful Presence'
}

predicted_cr = predict_challenge_rating(new_monster)
print(f"Predicted Challenge Rating: {predicted_cr}")


Predicted Challenge Rating: 22.35


In [14]:
# Manual CR Calc Info
def calculate_manual_cr(monster_attributes):
    """
    Calculate the manual CR for a monster using its attributes.
    """
    # Extract relevant attributes
    hit_points = monster_attributes.get('hit_points', 0)
    armor_class = monster_attributes.get('armor_class', 0)
    attack_bonus = monster_attributes.get('attack_bonus', 0)
    damage_per_round = monster_attributes.get('damage_per_round', 0)
    save_dc = monster_attributes.get('save_dc', 0)
    
    # Defensive CR calculation
    if hit_points < 1:
        raise ValueError("Hit Points must be greater than zero for manual CR calculation.")
    base_defensive_cr = hit_points
    if armor_class > 0:
        base_defensive_cr += armor_class  # Incorporate AC scaling
    
    # Offensive CR calculation
    base_offensive_cr = attack_bonus + damage_per_round
    if save_dc > 0:
        base_offensive_cr += save_dc  # Incorporate DC scaling
    
    # Combine defensive and offensive CRs
    manual_cr = (base_defensive_cr + base_offensive_cr) / 2
    
    return manual_cr



Section 2/ Option 2
Using the Kaggle Dataset

Read in the CSV files from the kaggle dataset found here: https://www.kaggle.com/datasets/shadowtime2000/dungeons-dragons?select=monsters.csv


In [13]:
import pandas as pd


# Read each CSV into a separate DataFrame
classes_df = pd.read_csv('Resources/classes.csv')
equipment_df = pd.read_csv('Resources/equipment.csv')
monsters_df = pd.read_csv('Resources/monsters.csv')
races_df = pd.read_csv('Resources/races.csv')
spells_df = pd.read_csv('Resources/spells.csv')

In [14]:
# Check classes dataframe
classes_df.head()


,Unnamed: 0,_id,index,name,hit_die,proficiency_choices,proficiencies,saving_throws,starting_equipment,class_levels,subclasses,url,spellcasting
0,0,5f2329a70b1bb138c5940d30,barbarian,Barbarian,12,"[{'choose': 2, 'type': 'proficiencies', 'from'...","[{'name': 'Light armor', 'url': '/api/proficie...","[{'name': 'STR', 'url': '/api/ability-scores/s...","{'url': '/api/starting-equipment/1', 'class': ...","{'url': '/api/classes/barbarian/levels', 'clas...","[{'url': '/api/subclasses/berserker', 'name': ...",/api/classes/barbarian,NaN
1,1,5f2329a70b1bb138c5940d31,bard,Bard,8,"[{'choose': 3, 'type': 'proficiencies', 'from'...","[{'name': 'Light armor', 'url': '/api/proficie...","[{'name': 'DEX', 'url': '/api/ability-scores/d...","{'url': '/api/starting-equipment/2', 'class': ...","{'url': '/api/classes/bard/levels', 'class': '...","[{'url': '/api/subclasses/lore', 'name': 'Lore'}]",/api/classes/bard,"{'url': '/api/spellcasting/bard', 'class': 'Ba..."
2,2,5f2329a70b1bb138c5940d32,cleric,Cleric,8,"[{'choose': 2, 'type': 'proficiencies', 'from'...","[{'name': 'Light armor', 'url': '/api/proficie...","[{'name': 'WIS', 'url': '/api/ability-scores/w...","{'url': '/api/starting-equipment/3', 'class': ...","{'url': '/api/classes/cleric/levels', 'class':...","[{'url': '/api/subclasses/life', 'name': 'Life'}]",/api/classes/cleric,"{'url': '/api/spellcasting/cleric', 'class': '..."
3,3,5f2329a70b1bb138c5940d34,druid,Druid,8,"[{'choose': 2, 'type': 'proficiencies', 'from'...","[{'name': 'Light armor', 'url': '/api/proficie...","[{'name': 'INT', 'url': '/api/ability-scores/i...","{'url': '/api/starting-equipment/4', 'class': ...","{'url': '/api/classes/druid/levels', 'class': ...","[{'url': '/api/subclasses/land', 'name': 'Land'}]",/api/classes/druid,"{'url': '/api/spellcasting/druid', 'class': 'D..."
4,4,5f2329a70b1bb138c5940d33,fighter,Fighter,10,"[{'choose': 2, 'type': 'proficiencies', 'from'...","[{'name': 'All armor', 'url': '/api/proficienc...","[{'name': 'STR', 'url': '/api/ability-scores/s...","{'url': '/api/starting-equipment/5', 'class': ...","{'url': '/api/classes/fighter/levels', 'class'...","[{'url': '/api/subclasses/champion', 'name': '...",/api/classes/fighter,NaN


In [15]:
# Check equipment dataframe
equipment_df.head()


,Unnamed: 0,_id,index,name,equipment_category,gear_category,cost,weight,url,desc,...,2h_damage,armor_category,armor_class,str_minimum,stealth_disadvantage,contents,speed,capacity,throw_range,special
0,0,5f2329a90b1bb138c5940de2,abacus,Abacus,"{'name': 'Adventuring Gear', 'url': '/api/equi...",Standard Gear,"{'quantity': 2, 'unit': 'gp'}",2.0,/api/equipment/abacus,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5f2329a90b1bb138c5940de3,acid-vial,Acid (vial),"{'name': 'Adventuring Gear', 'url': '/api/equi...",Standard Gear,"{'quantity': 25, 'unit': 'gp'}",1.0,/api/equipment/acid-vial,"['As an action, you can splash the contents of...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5f2329a90b1bb138c5940de4,alchemists-fire-flask,Alchemist's fire (flask),"{'name': 'Adventuring Gear', 'url': '/api/equi...",Standard Gear,"{'quantity': 50, 'unit': 'gp'}",1.0,/api/equipment/alchemists-fire-flask,"['This sticky, adhesive fluid ignites when exp...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5f2329a90b1bb138c5940e51,alchemists-supplies,Alchemist's supplies,"{'name': 'Tools', 'url': '/api/equipment-categ...",NaN,"{'quantity': 50, 'unit': 'gp'}",8.0,/api/equipment/alchemists-supplies,"[""These special tools include the items needed...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5f2329a90b1bb138c5940dea,amulet,Amulet,"{'name': 'Adventuring Gear', 'url': '/api/equi...",Holy Symbol,"{'quantity': 5, 'unit': 'gp'}",1.0,/api/equipment/amulet,['A holy symbol is a representation of a god o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Check Monsters dataframe
monsters_df.head()


,Unnamed: 0,_id,index,name,size,type,subtype,alignment,armor_class,hit_points,...,condition_immunities,senses,languages,challenge_rating,special_abilities,actions,legendary_actions,url,reactions,other_speeds
0,0,5f2329ad0b1bb138c59411ee,aboleth,Aboleth,Large,aberration,NaN,lawful evil,17,135,...,[],"{'darkvision': '120 ft.', 'passive_perception'...","Deep Speech, telepathy 120 ft.",10.00,"[{'name': 'Amphibious', 'desc': 'The aboleth c...","[{'name': 'Multiattack', 'desc': 'The aboleth ...","[{'name': 'Detect', 'desc': 'The aboleth makes...",/api/monsters/aboleth,NaN,NaN
1,1,5f2329ad0b1bb138c59411ef,acolyte,Acolyte,Medium,humanoid,any race,any alignment,10,9,...,[],{'passive_perception': 12},any one language (usually Common),0.25,"[{'name': 'Spellcasting', 'desc': 'The acolyte...","[{'name': 'Club', 'desc': 'Melee Weapon Attack...",NaN,/api/monsters/acolyte,NaN,NaN
2,2,5f2329ad0b1bb138c59411f1,adult-black-dragon,Adult Black Dragon,Huge,dragon,NaN,chaotic evil,19,195,...,[],"{'blindsight': '60 ft.', 'darkvision': '120 ft...","Common, Draconic",14.00,"[{'name': 'Amphibious', 'desc': 'The dragon ca...","[{'name': 'Multiattack', 'desc': 'The dragon c...","[{'name': 'Detect', 'desc': 'The dragon makes ...",/api/monsters/adult-black-dragon,NaN,NaN
3,3,5f2329ad0b1bb138c59411f0,adult-blue-dragon,Adult Blue Dragon,Huge,dragon,NaN,lawful evil,19,225,...,[],"{'blindsight': '60 ft.', 'darkvision': '120 ft...","Common, Draconic",16.00,"[{'name': 'Legendary Resistance', 'desc': 'If ...","[{'name': 'Multiattack', 'desc': 'The dragon c...","[{'name': 'Detect', 'desc': 'The dragon makes ...",/api/monsters/adult-blue-dragon,NaN,NaN
4,4,5f2329ad0b1bb138c59411f2,adult-brass-dragon,Adult Brass Dragon,Huge,dragon,NaN,chaotic good,18,172,...,[],"{'blindsight': '60 ft.', 'darkvision': '120 ft...","Common, Draconic",13.00,"[{'name': 'Legendary Resistance', 'desc': 'If ...","[{'name': 'Multiattack', 'desc': 'The dragon c...",NaN,/api/monsters/adult-brass-dragon,NaN,NaN


In [17]:
# Check races dataframe

races_df.head()

,Unnamed: 0,_id,index,name,speed,ability_bonuses,alignment,age,size,size_description,starting_proficiencies,languages,language_desc,traits,trait_options,subraces,url,starting_proficiency_options,ability_bonus_options,language_options
0,0,5f2329af0b1bb138c59413c3,dragonborn,Dragonborn,30,"[{'name': 'STR', 'url': '/api/ability-scores/s...","Dragonborn tend to extremes, making a conscio...",Young dragonborn grow quickly. They walk hours...,Medium,"Dragonborn are taller and heavier than humans,...",[],"[{'url': '/api/languages/common', 'name': 'Com...","You can speak, read, and write Common and Drac...","[{'name': 'Draconic Ancestry', 'url': '/api/tr...","{'choose': 1, 'from': [{'name': 'Breath Weapon...",[],/api/races/dragonborn,NaN,NaN,NaN
1,1,5f2329af0b1bb138c59413c0,dwarf,Dwarf,30,"[{'name': 'CON', 'url': '/api/ability-scores/c...","Most dwarves are lawful, believing firmly in t...","Dwarves mature at the same rate as humans, but...",Medium,Dwarves stand between 4 and 5 feet tall and av...,"[{'url': '/api/proficiencies/battleaxes', 'nam...","[{'url': '/api/languages/common', 'name': 'Com...","You can speak, read, and write Common and Dwar...","[{'name': 'Darkvision', 'url': '/api/traits/da...",NaN,"[{'url': '/api/subraces/hill-dwarf', 'name': '...",/api/races/dwarf,"{'choose': 1, 'type': 'proficiencies', 'from':...",NaN,NaN
2,2,5f2329af0b1bb138c59413bf,elf,Elf,30,"[{'name': 'DEX', 'url': '/api/ability-scores/d...","Elves love freedom, variety, and self-expressi...",Although elves reach physical maturity at abou...,Medium,Elves range from under 5 to over 6 feet tall a...,[{'url': '/api/proficiencies/skill-perception'...,"[{'url': '/api/languages/common', 'name': 'Com...","You can speak, read, and write Common and Elvi...","[{'name': 'Darkvision', 'url': '/api/traits/da...",NaN,"[{'url': '/api/subraces/high-elf', 'name': 'Hi...",/api/races/elf,NaN,NaN,NaN
3,3,5f2329af0b1bb138c59413c4,gnome,Gnome,25,"[{'name': 'INT', 'url': '/api/ability-scores/i...",Gnomes are most often good. Those who tend tow...,"Gnomes mature at the same rate humans do, and...",Small,Gnomes are between 3 and 4 feet tall and avera...,[],"[{'url': '/api/languages/common', 'name': 'Com...","You can speak, read, and write Common and Gnom...","[{'name': 'Darkvision', 'url': '/api/traits/da...",NaN,"[{'url': '/api/subraces/rock-gnome', 'name': '...",/api/races/gnome,NaN,NaN,NaN
4,4,5f2329af0b1bb138c59413c5,half-elf,Half-Elf,30,"[{'name': 'CHA', 'url': '/api/ability-scores/c...",Half-elves share the chaotic bent of their elv...,Half-elves mature at the same rate humans do a...,Medium,"Half-elves are about the same size as humans, ...",[],"[{'name': 'Common', 'url': '/api/languages/com...","You can speak, read, and write Common, Elvish,...","[{'name': 'Darkvision', 'url': '/api/traits/da...",NaN,[],/api/races/half-elf,NaN,"{'choose': 2, 'type': 'ability_bonuses', 'from...","{'choose': 1, 'type': 'languages', 'from': [{'..."


In [18]:
# Check spells dataframe
spells_df.head()

,Unnamed: 0,_id,index,name,desc,higher_level,range,components,material,ritual,...,level,attack_type,damage,school,classes,subclasses,url,dc,heal_at_slot_level,area_of_effect
0,0,5f2329b00b1bb138c594140a,acid-arrow,Acid Arrow,['A shimmering green arrow streaks toward a ta...,['When you cast this spell using a spell slot ...,90 feet,"['V', 'S', 'M']",Powdered rhubarb leaf and an adder's stomach.,False,...,2,ranged,"{'damage_type': {'name': 'Acid', 'url': '/api/...","{'name': 'Evocation', 'url': '/api/magic-schoo...","[{'name': 'Wizard', 'url': '/api/classes/wizar...","[{'name': 'Lore', 'url': '/api/subclasses/lore...",/api/spells/acid-arrow,NaN,NaN,NaN
1,1,5f2329b00b1bb138c5941409,acid-splash,Acid Splash,['You hurl a bubble of acid. Choose one creatu...,NaN,60 feet,"['V', 'S']",NaN,False,...,0,NaN,"{'damage_type': {'name': 'Acid', 'url': '/api/...","{'name': 'Conjuration', 'url': '/api/magic-sch...","[{'name': 'Sorcerer', 'url': '/api/classes/sor...","[{'name': 'Lore', 'url': '/api/subclasses/lore'}]",/api/spells/acid-splash,"{'dc_type': {'name': 'DEX', 'url': '/api/abili...",NaN,NaN
2,2,5f2329b00b1bb138c594140c,aid,Aid,"[""Your spell bolsters your allies with toughne...","[""When you cast this spell using a spell slot ...",30 feet,"['V', 'S', 'M']",A tiny strip of white cloth.,False,...,2,NaN,NaN,"{'name': 'Abjuration', 'url': '/api/magic-scho...","[{'name': 'Cleric', 'url': '/api/classes/cleri...","[{'name': 'Lore', 'url': '/api/subclasses/lore'}]",/api/spells/aid,NaN,"{'2': '5', '3': '10', '4': '15', '5': '20', '6...",NaN
3,3,5f2329b00b1bb138c594140b,alarm,Alarm,"[""You set an alarm against unwanted intrusion....",NaN,30 feet,"['V', 'S', 'M']",A tiny bell and a piece of fine silver wire.,True,...,1,NaN,NaN,"{'name': 'Abjuration', 'url': '/api/magic-scho...","[{'name': 'Ranger', 'url': '/api/classes/range...","[{'name': 'Lore', 'url': '/api/subclasses/lore'}]",/api/spells/alarm,NaN,NaN,"{'type': 'cube', 'size': 20}"
4,4,5f2329b00b1bb138c594140d,alter-self,Alter Self,['You assume a different form. When you cast t...,NaN,Self,"['V', 'S']",NaN,False,...,2,NaN,NaN,"{'name': 'Transmutation', 'url': '/api/magic-s...","[{'name': 'Sorcerer', 'url': '/api/classes/sor...","[{'name': 'Lore', 'url': '/api/subclasses/lore'}]",/api/spells/alter-self,NaN,NaN,NaN


In [19]:
# Add them to a dictionary
df_dict = {
    "classes": classes_df,
    "equipment": equipment_df,
    "races": races_df,
    "spells": spells_df,
    "monsters": monsters_df
}


In [ ]:
# Obtain information for each dataframe 
# Start with classes

print("--- classes_df ---")
print(classes_df.info())  # Overview of columns, data types, and non-null counts
print("Null values:\n", classes_df.isnull().sum())  # Null values per column
print("Data types:\n", classes_df.dtypes)  # Data types of columns
print("First 5 rows:\n", classes_df.head())  # Preview the first 5 rows
print()


--- classes_df ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           12 non-null     int64 
 1   _id                  12 non-null     object
 2   index                12 non-null     object
 3   name                 12 non-null     object
 4   hit_die              12 non-null     int64 
 5   proficiency_choices  12 non-null     object
 6   proficiencies        12 non-null     object
 7   saving_throws        12 non-null     object
 8   starting_equipment   12 non-null     object
 9   class_levels         12 non-null     object
 10  subclasses           12 non-null     object
 11  url                  12 non-null     object
 12  spellcasting         8 non-null      object
dtypes: int64(2), object(11)
memory usage: 1.3+ KB
None
Null values:
 Unnamed: 0             0
_id                    0
index                  

In [ ]:
# Make it a text file so the information is more easily read
with open('classes_info.txt', 'w') as f:
    f.write("--- classes_df ---\n")
    f.write(str(classes_df.info()) + "\n")
    f.write("Null values:\n" + str(classes_df.isnull().sum()) + "\n")
    f.write("Data types:\n" + str(classes_df.dtypes) + "\n")
    f.write("First 5 rows:\n" + str(classes_df.head()) + "\n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           12 non-null     int64 
 1   _id                  12 non-null     object
 2   index                12 non-null     object
 3   name                 12 non-null     object
 4   hit_die              12 non-null     int64 
 5   proficiency_choices  12 non-null     object
 6   proficiencies        12 non-null     object
 7   saving_throws        12 non-null     object
 8   starting_equipment   12 non-null     object
 9   class_levels         12 non-null     object
 10  subclasses           12 non-null     object
 11  url                  12 non-null     object
 12  spellcasting         8 non-null      object
dtypes: int64(2), object(11)
memory usage: 1.3+ KB


In [24]:
# Repeat for Monsters
with open('monsters_info.txt', 'w') as f:
    f.write("--- monsters_df ---\n")
    f.write(str(monsters_df.info()) + "\n")
    f.write("Null values:\n" + str(monsters_df.isnull().sum()) + "\n")
    f.write("Data types:\n" + str(monsters_df.dtypes) + "\n")
    f.write("First 5 rows:\n" + str(monsters_df.head()) + "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              322 non-null    int64  
 1   _id                     322 non-null    object 
 2   index                   322 non-null    object 
 3   name                    322 non-null    object 
 4   size                    322 non-null    object 
 5   type                    322 non-null    object 
 6   subtype                 64 non-null     object 
 7   alignment               322 non-null    object 
 8   armor_class             322 non-null    int64  
 9   hit_points              322 non-null    int64  
 10  hit_dice                322 non-null    object 
 11  speed                   322 non-null    object 
 12  strength                322 non-null    int64  
 13  dexterity               322 non-null    int64  
 14  constitution            322 non-null    in

In [25]:
# Repeat for equipment
with open('equipment_info.txt', 'w') as f:
    f.write("--- equipment_df ---\n")
    f.write(str(equipment_df.info()) + "\n")
    f.write("Null values:\n" + str(equipment_df.isnull().sum()) + "\n")
    f.write("Data types:\n" + str(equipment_df.dtypes) + "\n")
    f.write("First 5 rows:\n" + str(equipment_df.head()) + "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            231 non-null    int64  
 1   _id                   231 non-null    object 
 2   index                 231 non-null    object 
 3   name                  231 non-null    object 
 4   equipment_category    231 non-null    object 
 5   gear_category         110 non-null    object 
 6   cost                  231 non-null    object 
 7   weight                209 non-null    float64
 8   url                   231 non-null    object 
 9   desc                  104 non-null    object 
 10  tool_category         31 non-null     object 
 11  vehicle_category      40 non-null     object 
 12  quantity              4 non-null      float64
 13  weapon_category       37 non-null     object 
 14  weapon_range          37 non-null     object 
 15  category_range        3

In [26]:
# Repeat for races
with open('races_info.txt', 'w') as f:
    f.write("--- races_df ---\n")
    f.write(str(races_df.info()) + "\n")
    f.write("Null values:\n" + str(races_df.isnull().sum()) + "\n")
    f.write("Data types:\n" + str(races_df.dtypes) + "\n")
    f.write("First 5 rows:\n" + str(races_df.head()) + "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    9 non-null      int64 
 1   _id                           9 non-null      object
 2   index                         9 non-null      object
 3   name                          9 non-null      object
 4   speed                         9 non-null      int64 
 5   ability_bonuses               9 non-null      object
 6   alignment                     9 non-null      object
 7   age                           9 non-null      object
 8   size                          9 non-null      object
 9   size_description              9 non-null      object
 10  starting_proficiencies        9 non-null      object
 11  languages                     9 non-null      object
 12  language_desc                 9 non-null      object
 13  traits                  

In [27]:
# Repeat for spells
with open('spells_info.txt', 'w') as f:
    f.write("--- spells_df ---\n")
    f.write(str(spells_df.info()) + "\n")
    f.write("Null values:\n" + str(spells_df.isnull().sum()) + "\n")
    f.write("Data types:\n" + str(spells_df.dtypes) + "\n")
    f.write("First 5 rows:\n" + str(spells_df.head()) + "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          319 non-null    int64 
 1   _id                 319 non-null    object
 2   index               319 non-null    object
 3   name                319 non-null    object
 4   desc                319 non-null    object
 5   higher_level        87 non-null     object
 6   range               319 non-null    object
 7   components          319 non-null    object
 8   material            184 non-null    object
 9   ritual              319 non-null    bool  
 10  duration            319 non-null    object
 11  concentration       319 non-null    bool  
 12  casting_time        319 non-null    object
 13  level               319 non-null    int64 
 14  attack_type         20 non-null     object
 15  damage              69 non-null     object
 16  school              319 no